In [ ]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requ

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
hugging_face_dataset = "knkarthick/dialogsum"

In [ ]:
dataset = load_dataset(hugging_face_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
indices = [40, 600]
seperation_line = "-".join("" for x in range(100))

for i, index in enumerate(indices):
  print(seperation_line)
  print("Example ", i+1)
  print("Prompt Dialogue : ")
  print(dataset['test'][index]['dialogue'])
  print(seperation_line)
  print("Baseline human summary:")
  print(dataset['test'][index]['summary'])
  print()

---------------------------------------------------------------------------------------------------
Example  1
Prompt Dialogue : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
Example  2
Prompt Dialogue : 
#Person1#: Oh, I'm starving. It's my first time to China. And I'd like to try some real Chinese cuisine. What would you recommend?
#Person2#: Well, depends. You see, ther

In [ ]:
model = "google/flan-t5-base"

prediction_model = AutoModelForSeq2SeqLM.from_pretrained(model)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
sentence = "What time is it, Sanidhya ?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded= tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True
)

In [ ]:
print("ENCODED_SENTENCE : ")
print(sentence_encoded["input_ids"][0])
print("DECODED _ SENTENCE : ")
print(sentence_decoded)

ENCODED_SENTENCE : 
tensor([ 363,   97,   19,   34,    6, 1051,   23,   26,  107,   63,    9,    3,
          58,    1])
DECODED _ SENTENCE : 
What time is it, Sanidhya?


In [ ]:
for i, index in enumerate(indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      prediction_model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )


  print(seperation_line)
  print("Exmaple ", i+1)
  print(f"Input Prompt : {dialogue}")
  print(seperation_line)
  print(f"Baseline Human Summary : {summary}")
  print(seperation_line)
  print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  1
Input Prompt : #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary : #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generated output : Person1: It's ten to nine.
---------------------------------------------------------------------------------------------------
Exmaple  2
Input Prompt : #Person1#: Oh, I'm starvin

In [ ]:
for i, index in enumerate(indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary :
  """

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      prediction_model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )


  print(seperation_line)
  print("Exmaple ", i+1)
  print(f"Input Prompt : {prompt}")
  print(seperation_line)
  print(f"Baseline Human Summary : {summary}")
  print(seperation_line)
  print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  1
Input Prompt : 
  Summarize the following conversation.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
  Summary : 
  
---------------------------------------------------------------------------------------------------
Baseline Human Summary : #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generated output : Person1: It's ten to nine.
---------------------------------------------------------------------------------------------

Zero Shot with prompt template

In [ ]:
for i, index in enumerate(indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Dialogue:
  {dialogue}
  What was going on ?
  """

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      prediction_model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )


  print(seperation_line)
  print("Exmaple ", i+1)
  print(f"Input Prompt : {prompt}")
  print(seperation_line)
  print(f"Baseline Human Summary : {summary}")
  print(seperation_line)
  print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  1
Input Prompt : 
  Dialogue:
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
  What was going on ?
  
---------------------------------------------------------------------------------------------------
Baseline Human Summary : #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generated output : Person1: It's ten to nine.
---------------------------------------------------------------------------------------------------
Exmaple  2
I

One Shto inference

In [ ]:
def oneshot_prompt_maker(example_indices, example_index_to_summarize):
  prompt = ''
  for index in example_indices:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt += f"""
    Doalogue:
    {dialogue}
    What was going on ?
    {summary}
    """

  dialogue = dataset['test'][example_index_to_summarize]['dialogue']
  prompt += f"""
  Dialogue:
  {dialogue}

  What was going on ?
  """

  return prompt


In [ ]:

summary = dataset['test'][index]['summary']

inputs = tokenizer(oneshot_prompt_maker([40],600), return_tensors='pt')
output = tokenizer.decode(
    prediction_model.generate(
        inputs["input_ids"],
        max_new_tokens = 100,
    )[0],
    skip_special_tokens = True
)


print(seperation_line)
print("Exmaple ", i+1)
print(f"Input Prompt : {prompt}")
print(seperation_line)
print(f"Baseline Human Summary : {summary}")
print(seperation_line)
print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  2
Input Prompt : 
  Dialogue:
  #Person1#: Oh, I'm starving. It's my first time to China. And I'd like to try some real Chinese cuisine. What would you recommend?
#Person2#: Well, depends. You see, there are eight famous Chinese food cuisines, for instance, Sichuan cuisine and Hunan cuisine.
#Person1#: There're all spicy or hot of heard.
#Person2#: That's right. If you have hot dishes, you can try some.
#Person1#: I cannot have it. Last time I had some in the US. It almost killed me.
#Person2#: And there are Cantonese and Kiangsu cuisines. Most people like them.
#Person1#: Oh I'd like to try the Cantonese one. Where is it? Is it far?
#Person2#: The one I know is about half an hour to go.
#Person1#: Oh. That's too far away. I am really starvig. Do you have restaurant in your hotel?
#Person2#: Oh sorry, we don't. But I know one nearby.
#Person1#: What type?
#Person2#: It's Beijing

Few Shot Inference

In [ ]:

summary = dataset['test'][index]['summary']

inputs = tokenizer(oneshot_prompt_maker([40,80,120],600), return_tensors='pt')
output = tokenizer.decode(
    prediction_model.generate(
        inputs["input_ids"],
        max_new_tokens = 100,
    )[0],
    skip_special_tokens = True
)


print(seperation_line)
print("Exmaple ", i+1)
print(f"Input Prompt : {prompt}")
print(seperation_line)
print(f"Baseline Human Summary : {summary}")
print(seperation_line)
print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  2
Input Prompt : 
  Dialogue:
  #Person1#: Oh, I'm starving. It's my first time to China. And I'd like to try some real Chinese cuisine. What would you recommend?
#Person2#: Well, depends. You see, there are eight famous Chinese food cuisines, for instance, Sichuan cuisine and Hunan cuisine.
#Person1#: There're all spicy or hot of heard.
#Person2#: That's right. If you have hot dishes, you can try some.
#Person1#: I cannot have it. Last time I had some in the US. It almost killed me.
#Person2#: And there are Cantonese and Kiangsu cuisines. Most people like them.
#Person1#: Oh I'd like to try the Cantonese one. Where is it? Is it far?
#Person2#: The one I know is about half an hour to go.
#Person1#: Oh. That's too far away. I am really starvig. Do you have restaurant in your hotel?
#Person2#: Oh sorry, we don't. But I know one nearby.
#Person1#: What type?
#Person2#: It's Beijing

Adjusting Parameters

In [ ]:

summary = dataset['test'][index]['summary']

generation_config = GenerationConfig(max_new_tokens=100, temprature=0.9)
# generation_config = GenerationConfig(max_new_tokens=100, temprature=0.5)

inputs = tokenizer(oneshot_prompt_maker([40,80,120],600), return_tensors='pt')
output = tokenizer.decode(
    prediction_model.generate(
        inputs["input_ids"],
      generation_config
    )[0],
    skip_special_tokens = True
)


print(seperation_line)
print("Exmaple ", i+1)
print(f"Input Prompt : {prompt}")
print(seperation_line)
print(f"Baseline Human Summary : {summary}")
print(seperation_line)
print(f"Model Generated output : {output}")

---------------------------------------------------------------------------------------------------
Exmaple  2
Input Prompt : 
  Dialogue:
  #Person1#: Oh, I'm starving. It's my first time to China. And I'd like to try some real Chinese cuisine. What would you recommend?
#Person2#: Well, depends. You see, there are eight famous Chinese food cuisines, for instance, Sichuan cuisine and Hunan cuisine.
#Person1#: There're all spicy or hot of heard.
#Person2#: That's right. If you have hot dishes, you can try some.
#Person1#: I cannot have it. Last time I had some in the US. It almost killed me.
#Person2#: And there are Cantonese and Kiangsu cuisines. Most people like them.
#Person1#: Oh I'd like to try the Cantonese one. Where is it? Is it far?
#Person2#: The one I know is about half an hour to go.
#Person1#: Oh. That's too far away. I am really starvig. Do you have restaurant in your hotel?
#Person2#: Oh sorry, we don't. But I know one nearby.
#Person1#: What type?
#Person2#: It's Beijing